PYTHON-11. Базовые приёмы работы с данными в Pandas

1. Введение

In [ ]:
import pandas as pd
melb_data = pd.read_csv('data_11/melb_data_ps.csv', sep=',')
melb_data.head()


2. Базовые операции со столбцами DataFrame

In [ ]:
import pandas as pd
melb_data = pd.read_csv('data_11/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()
#melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)# Удалим столбцы index и Coordinates
melb_df.drop(['index','Coordinates'],axis=1,inplace=True) # Альтернативный вариант
#melb_df.head()
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom'] #создадим переменную total_rooms, в которой будем хранить общее количество комнат в здании. Для этого выполним сложение столбцов с количеством комнат, ванн и спален
#display(total_rooms)
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms #введём признак MeanRoomsSquare, который соответствует средней площади одной комнаты для каждого объекта. Для этого разделим площадь здания на полученное ранее общее количество комнат
#display(melb_df['MeanRoomsSquare'])

diff_area = melb_df['BuildingArea'] - melb_df['Landsize'] #AreaRatio, коэффициент соотношения площади здания (BuildingArea) и площади участка (Landsize). Для этого разницу двух площадей поделим на их сумму
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
display(melb_df['AreaRatio'])


Задание 2.3 (External resource)

In [10]:
import pandas as pd
customer_df = pd.DataFrame({
            'number': [0, 1, 2, 3, 4],
            'cust_id': [128, 1201, 9832, 4392, 7472],
            'cust_age': [13, 21, 19, 21, 60],
            'cust_sale': [0, 0, 0.2, 0.15, 0.3],
            'cust_year_birth': [2008, 2000, 2002, 2000, 1961],
            'cust_order': [1400, 14142, 900, 1240, 8430]
        })
def delete_columns(df, col=[]):
    for cc in col:
        if cc not in df.columns:
            return None
    return df.drop(col, axis=1)

Задание 2.4

In [14]:
countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})
countries_df['density'] = countries_df['population']*1000000 / countries_df['square']
print(round(countries_df['density'].mean(), 2))

84.93


3. Работа с датами в DataFrame

In [109]:
#display(melb_df['Date'])
#display(melb_df.loc[2,'Date'])
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
#display(melb_df['Date'])
years_sold = melb_df['Date'].dt.year
#print(years_sold)
#print('Min year sold:', years_sold.min())
#print('Max year sold:', years_sold.max())
#print('Mode year sold:', years_sold.mode()[0])
melb_df['MonthSale'] = melb_df['Date'].dt.month
#display(melb_df['MonthSale'].value_counts(normalize=True))
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
#display(delta_days)
#display(delta_days.dt.days)
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
#display(melb_df['AgeBuilding'])
melb_df = melb_df.drop('YearBuilt', axis=1)

melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count=melb_df[     (melb_df['WeekdaySale']==5) |  (melb_df['WeekdaySale']==6)      ].shape[0] #Задание 3.3
display(weekend_count)


12822

In [59]:
import pandas as pd
ufo_data = pd.read_csv('https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/ufo.csv', sep=',')
ufo_df = ufo_data.copy()
#display(ufo_df.head())
#display(ufo_df.shape)
#display(ufo_df.info)
#display(ufo_df.describe)

ufo_df['Time'] = pd.to_datetime(ufo_df['Time'])
ufo_df['Year'] = ufo_df['Time'].dt.year
ufo_df['Year'].mode()[0] #Задание 3.4 

ufo_df['Date'] = ufo_df['Time'].dt.date
print(ufo_df [ufo_df['State']=='NV']['Date'].diff().dt.days.mean()) #Задание 3.5


68.92932862190813


4. Создание и преобразование столбцов с помощью функций

In [108]:
#print(melb_df['Address'].nunique())
#print(melb_df['Address'].shape)
#print(melb_df['Address'].loc[177])
#print(melb_df['Address'].loc[1812])
#print(melb_df['Address'].loc[9001])
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type
street_types = melb_df['Address'].apply(get_street_type)
#display(street_types)
print(street_types.nunique())
display(street_types.value_counts())
popular_stypes =street_types.value_counts().nlargest(10).index
print(popular_stypes)
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
#display(melb_df['StreetType'])
#print(melb_df['StreetType'].nunique())#Посмотрим на результирующее число уникальных подтипов:
# 11

56


Address
St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Co

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object', name='Address')


ЗАДАНИЕ 4.2

In [106]:
#display(melb_df['Date'])
#display(melb_df.loc[2,'Date'])
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
#display(melb_df['Date'])
years_sold = melb_df['Date'].dt.year
#print(years_sold)
#print('Min year sold:', years_sold.min())
#print('Max year sold:', years_sold.max())
#print('Mode year sold:', years_sold.mode()[0])
melb_df['MonthSale'] = melb_df['Date'].dt.month
#display(melb_df['MonthSale'].value_counts(normalize=True))
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
#display(delta_days)
#display(delta_days.dt.days)
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
#display(melb_df['AgeBuilding'])
melb_df = melb_df.drop('YearBuilt', axis=1)

melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count=melb_df[     (melb_df['WeekdaySale']==5) |  (melb_df['WeekdaySale']==6)      ].shape[0] #Задание 3.3
display(weekend_count)
def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else: 
        return 0
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
print(round(melb_df[melb_df['Weekend']==1]['Price'].mean(), 2))

12822

1081198.64


Задание 4.3

In [105]:
popular_SellerG =melb_df['SellerG'].value_counts().nlargest(49).index
#print(popular_SellerG)
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_SellerG else 'other')
print(melb_df['SellerG'].shape)
print(melb_df['SellerG'].nunique())


mask_11=melb_df[melb_df['SellerG']=='Nelson']['Price'].min()
mask_22=melb_df[melb_df['SellerG']=='other']['Price'].min()

print (mask_11)
print (mask_22)
print(round(mask_11/mask_22,2))

print(round(melb_df[melb_df['SellerG']=='Nelson']['Price'].min()/melb_df[melb_df['SellerG']=='other']['Price'].min(),2))


(13580,)
50
170000.0
131000.0
1.3
1.3


Задание 4.4 (External resource)

In [101]:
def get_experience(arg):
    month_key_words = ['месяц', 'месяцев', 'месяца']
    year_key_words = ['год', 'лет', 'года']
    args_splited = arg.split(' ')
    month = 0
    year = 0
    for i in range(len(args_splited)):
        if args_splited[i] in month_key_words:
            month = args_splited[i-1]
        if args_splited[i] in year_key_words:
            year = args_splited[i-1]
    return int(year)*12 + int(month)

5. Тип данных Category

In [119]:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
#display(unique_counts)
#display(melb_df.info())
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
#display(melb_df.info())
#print(melb_df['Regionname'].cat.categories)
#display(melb_df['Regionname'].cat.codes)
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
#display(melb_df['Type'])
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

0     unit
1    house
2      NaN
3      NaN
4    house
dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

6.закрепление знаний

In [203]:
import pandas as pd
citibike_tripdata = pd.read_csv('data_11/citibike-tripdata.csv', sep=',')
citibike_tripdata.head()
#display(citibike_tripdata.shape)
#display(citibike_tripdata['usertype'].mode()[0])
#display(citibike_tripdata.info()) #Задание 6.1 Задание 6.2
#display(citibike_tripdata.describe())
#display(citibike_tripdata['start station id'].mode()[0])#Задание 6.3
#display(citibike_tripdata['bikeid'].mode()[0])#Задание 6.4

#display(citibike_tripdata['usertype'].mode()[0])#Задание 6.5
#count_mode_user = citibike_tripdata[citibike_tripdata['usertype'] == citibike_tripdata['usertype'].mode()[0]].shape[0]
#print(round(count_mode_user / citibike_tripdata.shape[0], 2))

#male_count = citibike_tripdata[citibike_tripdata['gender'] == 1].shape[0]
#female_count = citibike_tripdata[citibike_tripdata['gender'] == 0].shape[0]
#display(male_count)
#display(female_count)
#print(max([male_count, female_count])) #Задание 6.6

#display(citibike_tripdata.head())
#display(citibike_tripdata['start station id'].nunique()==citibike_tripdata['end station id'].nunique()) #Задание 6.7 A
#display(  (2018-citibike_tripdata['birth year'].max())==10   )  #Задание 6.7 B
#display(citibike_tripdata['start station name'].value_counts())  #Задание 6.7 C
#display(citibike_tripdata['end station name'].value_counts())  #Задание 6.7 D

citibike_tripdata_df = citibike_tripdata.copy()
citibike_tripdata_df.drop(['start station id','end station id'],axis=1,inplace=True)
#display(citibike_tripdata_df.shape[1]) ##Задание 6.8

citibike_tripdata_df['age'] = 2018-citibike_tripdata['birth year']
citibike_tripdata_df.drop(['birth year'],axis=1,inplace=True)
#display(citibike_tripdata_df['age'])
#display(citibike_tripdata_df.head())
#display(   citibike_tripdata_df  [citibike_tripdata_df['age']>60]   .shape[0]  ) #Задание 6.8

citibike_tripdata_df['trip duration']=pd.to_datetime(citibike_tripdata_df['stoptime'])-pd.to_datetime(citibike_tripdata_df['starttime'])
#display(citibike_tripdata_df['trip duration'][3]) #Задание 6.10

citibike_tripdata_df['starttime'] = pd.to_datetime(citibike_tripdata_df['starttime'])
weekday = citibike_tripdata_df['starttime'].dt.dayofweek
citibike_tripdata_df['weekend'] = weekday.apply(lambda x: 1 if x ==5 or x == 6 else 0)
#display(citibike_tripdata_df['weekend'])
#display(citibike_tripdata_df['weekend'].sum()) #Задание 6.10

def get_time_of_day(time):
    if 0 <= time <= 6:
        return 'night'
    elif 6 < time <= 12:
        return 'morning'
    elif 12 < time <= 18:
        return 'day'
    elif 18 < time <= 23:
        return 'evening'

citibike_tripdata_df['time_of_day'] = citibike_tripdata_df['starttime'].dt.hour.apply(get_time_of_day)
a = citibike_tripdata_df[citibike_tripdata_df['time_of_day'] == 'day'].shape[0]
b = citibike_tripdata_df[citibike_tripdata_df['time_of_day'] == 'night'].shape[0]
print(round(a / b))




9
